In [1]:
import numpy as np
import random
import numpy.linalg as lia
import pandas as pd
import h5py
import random
import math
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn import datasets
from sklearn.cluster import KMeans

#to plot images
import matplotlib.image as mpimg 
#to read images

import cv2 
#open CV library for Python
import numpy as np
import torch
import torchvision
from torchvision.ops import nms
import torchvision.transforms as transforms
from statistics import stdev

In [2]:
!pip install opencv-python==4.4.0.44
!pip install opencv-contrib-python==4.4.0.44
!pip list

     |████████████████████████████████| 52.3 MB 389 kB/s  eta 0:00:01    |███████████▏                    | 18.2 MB 3.2 MB/s eta 0:00:11     |████████████████████████        | 39.2 MB 70.1 MB/s eta 0:00:01     |████████████████████████▉       | 40.6 MB 70.1 MB/s eta 0:00:01
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.4.0.46
    Uninstalling opencv-python-4.4.0.46:
      Successfully uninstalled opencv-python-4.4.0.46
     |████████████████████████████████| 64.0 MB 395 kB/s  eta 0:00:01   |██▍                             | 4.8 MB 2.9 MB/s eta 0:00:21     |████▎                           | 8.6 MB 2.9 MB/s eta 0:00:20     |████████▍                       | 16.8 MB 41.2 MB/s eta 0:00:02     |█████████▎                      | 18.7 MB 41.2 MB/s eta 0:00:02     |██████████                      | 19.9 MB 41.2 MB/s eta 0:00:02     |████████████▉                   | 25.6 MB 41.2 MB/s eta 0:00:01
Package                            Version
---------------

sortedcollections                  1.2.1
sortedcontainers                   2.2.2
soupsieve                          2.0.1
Sphinx                             3.1.2
sphinxcontrib-applehelp            1.0.2
sphinxcontrib-devhelp              1.0.2
sphinxcontrib-htmlhelp             1.0.3
sphinxcontrib-jsmath               1.0.1
sphinxcontrib-qthelp               1.0.3
sphinxcontrib-serializinghtml      1.1.4
sphinxcontrib-websupport           1.2.3
spyder                             4.1.4
spyder-kernels                     1.9.2
SQLAlchemy                         1.3.18
statsmodels                        0.11.1
sympy                              1.6.1
tables                             3.6.1
tblib                              1.6.0
terminado                          0.8.3
testpath                           0.4.4
threadpoolctl                      2.1.0
toml                               0.10.1
toolz                              0.10.0
torch                              1.7.0
torchvision 

In [3]:
def loadBGRImage(path):
    image = BGR(cv.imread(path))
    return image

def loadGreyImage(path):
    image = cv.imread(path, cv.IMREAD_GRAYSCALE)
    return image

def BGR(image):
    image = cv.cvtColor(image, cv.COLOR_RGB2BGR)
    return image

def displayGreyImage(image, imageName):
    plt.figure(figsize=(10, 10))
    plt.imshow(image, cmap = 'gray')
    plt.title(imageName)
    plt.show()

def displayGreyWindows(image, imageName):
    plt.figure(figsize=(3, 3))
    plt.imshow(image, cmap = 'gray')
    plt.title(imageName)
    plt.show()
    
def displayBGRImage(image, imageName, size):
    plt.figure(figsize=(size, size))
    plt.imshow(image)
    plt.title(imageName)
    plt.show()
    
def displayBGRImageLarge(image, imageName):
    plt.figure(figsize=(18, 18))
    plt.imshow(image)
    plt.title(imageName)
    plt.show()
    
def imageSideBySide(images, imageNames,size):
    row = np.ceil(len(images)/20)
    fig=plt.figure(figsize=(size, size/2))
    for i, image in enumerate(images):
        fig.add_subplot(row, 20, i+1)
        plt.imshow(image, cmap='gray')
        plt.title(imageNames[i])
    plt.show()

In [4]:
start = 25, 26
size = 14, 12

f = h5py.File('MNIST_synthetic.h5', 'r')

list(f.keys())

['test_dataset', 'train_dataset', 'train_labels']

In [5]:
train_dataset = np.squeeze(np.array(f["train_dataset"])).copy()
train_labels = np.squeeze(np.array(f["train_labels"])).copy()
test_dataset = np.squeeze(np.array(f["test_dataset"])).copy()

singulars_digits = []
singulars_labels = []

doubles_digits = []
doubles_labels = []

triples_digits = []
triples_labels = []

quadruples_digits = []
quadruples_labels = []

quintuples_digits = []
quintuples_labels = []


for i, labels in enumerate(train_labels):
    if labels[1] == 10:
        singulars_digits.append(train_dataset[i])
        singulars_labels.append(train_labels[i])
        
    if labels[1] != 10 and labels[2] == 10:
        doubles_digits.append(train_dataset[i])
        doubles_labels.append(train_labels[i])
        
    if labels[2] != 10 and labels[3] == 10:
        triples_digits.append(train_dataset[i])
        triples_labels.append(train_labels[i])
        
    if labels[3] != 10 and labels[4] == 10:
        quadruples_digits.append(train_dataset[i])
        quadruples_labels.append(train_labels[i])
        
    if labels[4] != 10:
        quintuples_digits.append(train_dataset[i])
        quintuples_labels.append(train_labels[i])

In [6]:
singulars_digits = np.array(singulars_digits)     
doubles_digits = np.array(doubles_digits)  
triples_digits = np.array(triples_digits)    
quadruples_digits = np.array(quadruples_digits)    
quintuples_digits = np.array(quintuples_digits)    

singulars_labels = np.array(singulars_labels).T[0]
doubles_labels = np.array(doubles_labels).T[0:2].T
triples_label = np.array(triples_labels).T[0:3].T
quadruples_label = np.array(quadruples_labels).T[0:4].T
quintuples_label = np.array(quintuples_labels).T[0:5].T

In [7]:
final_singulars_dataset = []
final_singulars_labels = list(singulars_labels)
for i, digit in enumerate(singulars_digits):
    final_singulars_dataset.append(digit[26:38,26:38])
    
final_doubles_dataset = []
final_doubles_labels = []
for i, digit in enumerate(doubles_digits):
    final_doubles_dataset.append(digit[26:38,20:32])
    final_doubles_dataset.append(digit[26:38,32:44])
    final_doubles_labels.append(doubles_labels[i][0])
    final_doubles_labels.append(doubles_labels[i][1])

final_triples_dataset = []
final_triples_labels = []
for i, digit in enumerate(triples_digits):
    final_triples_dataset.append(digit[26:38,14:26])
    final_triples_dataset.append(digit[26:38,26:38])
    final_triples_dataset.append(digit[26:38,38:50])
    final_triples_labels.append(triples_labels[i][0])
    final_triples_labels.append(triples_labels[i][1])
    final_triples_labels.append(triples_labels[i][2])
    
final_quadruples_dataset = []
final_quadruples_labels = []
for i, digit in enumerate(quadruples_digits):
    final_quadruples_dataset.append(digit[26:38,8:20])
    final_quadruples_dataset.append(digit[26:38,20:32])
    final_quadruples_dataset.append(digit[26:38,32:44])
    final_quadruples_dataset.append(digit[26:38,44:56])
    final_quadruples_labels.append(quadruples_labels[i][0])
    final_quadruples_labels.append(quadruples_labels[i][1])    
    final_quadruples_labels.append(quadruples_labels[i][2])
    final_quadruples_labels.append(quadruples_labels[i][3]) 
    
final_quintuples_dataset = []
final_quintuples_labels = []
for i, digit in enumerate(quintuples_digits):
    final_quintuples_dataset.append(digit[26:38,2:14])
    final_quintuples_dataset.append(digit[26:38,14:26])
    final_quintuples_dataset.append(digit[26:38,26:38])
    final_quintuples_dataset.append(digit[26:38,38:50])
    final_quintuples_dataset.append(digit[26:38,50:62])
    final_quintuples_labels.append(quintuples_labels[i][0])
    final_quintuples_labels.append(quintuples_labels[i][1])
    final_quintuples_labels.append(quintuples_labels[i][2])
    final_quintuples_labels.append(quintuples_labels[i][3])
    final_quintuples_labels.append(quintuples_labels[i][4])

In [8]:
merged_dataset = final_singulars_dataset + final_doubles_dataset + final_triples_dataset + final_quadruples_dataset + final_quintuples_dataset
merged_labels = final_singulars_labels + final_doubles_labels + final_triples_labels + final_quadruples_labels + final_quintuples_labels

In [9]:
count = 0

for digit in range(10):
    digit_count = list(merged_labels).count(digit)
    print(f"number of {digit}'s in train dataset: {digit_count}")
    count += digit_count

print("total number of digits in train dataset:", count)

number of 0's in train dataset: 16379
number of 1's in train dataset: 18819
number of 2's in train dataset: 17095
number of 3's in train dataset: 17220
number of 4's in train dataset: 16393
number of 5's in train dataset: 15275
number of 6's in train dataset: 16401
number of 7's in train dataset: 17611
number of 8's in train dataset: 16175
number of 9's in train dataset: 16518
total number of digits in train dataset: 167886


In [10]:
digitsTrainingSetSize = int(np.ceil(0.8 * len(merged_dataset)))
digitsValidationSetSize = int(len(merged_labels) - digitsTrainingSetSize)

xValidationSet = []
yValidationSet = []

for index, digit in enumerate(merged_dataset[0:digitsValidationSetSize]):
    xValidationSet.append(digit)
    yValidationSet.append(merged_labels[index])

xTrainingSet = []
yTrainingSet = []

start = len(xValidationSet)

for i, digit in enumerate(merged_dataset[start:]):
    xTrainingSet.append(digit)
    yTrainingSet.append(merged_labels[i+start])

## Finding all digits in an image

In [11]:
def find_digits(train_dataset, size):
    thresh_train_dataset = train_dataset.copy()
    thresh_train_dataset[thresh_train_dataset>30] = 255
    thresh_train_dataset[thresh_train_dataset!=255] = 0
    #displayGreyImage(thresh_train_dataset,"")
    i,j = np.where(thresh_train_dataset[:,:]!=0)
    #print(i)
    bottom_bound = np.min(i)
    top_bound = np.max(i)
    left_bound = np.min(j)
    right_bound = np.max(j)
    #print(top_bound, bottom_bound)

    h = top_bound-bottom_bound
    w = right_bound-left_bound

    bounded_train_dataset = train_dataset[bottom_bound-1:top_bound+2,left_bound-1:right_bound+2]
    thresh = bounded_train_dataset.copy()
    thresh[thresh>30] = 255
    thresh[thresh!=255] = 0
    #displayGreyImage(bounded_train_dataset,"")

    #ret, thresh = cv2.threshold(bounded_train_dataset, 30, 255, 0)
    #seg = np.where(np.any(thresh, axis=0)==0)
    seg = np.where(np.any(thresh, axis=0)==0)

    #print(seg)
    seg_list = np.asarray(seg)
    seg_list = seg_list[0]
    #print(seg_list)

    from statistics import stdev
    sd =0 # remove this and its return when done
    if len(seg_list)>2:
        # create a list of the gaps between the consecutive values
        gaps = [y - x for x, y in zip(seg_list[:-1], seg_list[1:])]
        # have python calculate the standard deviation for the gaps
        sd = stdev(gaps)
        #print(sd)

        # create a list of lists, put the first value of the source data in the first
        lists = [[seg_list[0]]]
        for x in seg_list[1:]:
            # if the gap from the current item to the previous is more than 1 SD
            # Note: the previous item is the last item in the last list
            # Note: the '> 1' is the part you'd modify to make it stricter or more relaxed
            if sd<1.5 and sd>0:
                if (x - lists[-1][-1]) / (sd+1e-18) > 1.5:
                    # then start a new list
                    lists.append([])
              # add the current item to the last list in the list
                lists[-1].append(x)
            elif sd==0:
                if (x - lists[-1][-1])>1:
                    lists.append([])
                lists[-1].append(x)
            else:
                if (x - lists[-1][-1]) / (sd+1e-18) > 0.8:
                  # then start a new list
                  lists.append([])
                # add the current item to the last list in the list
                lists[-1].append(x)

        splits = np.asarray([np.ceil(np.mean(lists[i])) for i in range(len(lists))]).astype(int)

    else:
        splits = np.asarray(seg_list)
      #print(splits)

    n_digits = len(splits)-1
    #digits = np.zeros(n_digits, )
    digits = []
    for i in range(n_digits):
        temp = bounded_train_dataset[:,splits[i]:splits[i+1]]
        # if temp is less than recommeded size first pad on left and then on both sides
        temp_padded = temp.copy()
        if (temp.shape[0] != size) or  (temp.shape[1] != size):
            diff_y = size-temp.shape[0]
            split_diff_y = diff_y//2
            remainder_diff_y = diff_y%2
            diff_x = size-temp.shape[1]
            split_diff_x = diff_x//2
            remainder_diff_x = diff_x%2
            #print(diff_y)
            #print(split_diff_y)
            temp_padded = np.pad(temp, ((split_diff_y+remainder_diff_y,split_diff_y),(split_diff_x+remainder_diff_x,split_diff_x)))

        digits.append(temp_padded)
      #displayGreyImage(train_dataset[bottom_bound-1:top_bound+2,left_bound-1:right_bound+2],train_labels[367])
    return digits, n_digits

In [12]:
def create_boxes_hacks(image, dig_centers):
    
    n_dig = len(dig_centers)
    digits = []
    if n_dig == 1:
        digits.append(image[26:38,26:38])
    elif n_dig == 2:
        digits.append(image[26:38,20:32])
        digits.append(image[26:38,32:44])
    elif n_dig == 3:
        digits.append(image[26:38,14:26])
        digits.append(image[26:38,26:38])
        digits.append(image[26:38,38:50])
    elif n_dig == 4:
        digits.append(image[26:38,8:20])
        digits.append(image[26:38,20:32])
        digits.append(image[26:38,32:44])
        digits.append(image[26:38,44:56])
    elif n_dig == 5:
        digits.append(image[26:38,2:14])
        digits.append(image[26:38,14:26])
        digits.append(image[26:38,26:38])
        digits.append(image[26:38,38:50])
        digits.append(image[26:38,50:62])

    return digits

In [13]:
import random as rng
rng.seed(12345)
def thresh_callback(src, val=30):
    threshold = val

    src_gray = cv2.blur(src, (1,1))

    #src_gray = bounding_box(src_gray,30)

    canny_output = cv2.Canny(src_gray, threshold, threshold * 2)
    
    
    contours, _ = cv2.findContours(canny_output, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    
    contours_poly = [None]*len(contours)
    boundRect = [None]*len(contours)
    centers = [None]*len(contours)
    radius = [None]*len(contours)
    for i, c in enumerate(contours):
        contours_poly[i] = cv2.approxPolyDP(c, 3, True)
        boundRect[i] = cv2.boundingRect(contours_poly[i])
        centers[i], radius[i] = cv2.minEnclosingCircle(contours_poly[i])
    

    arr_centers = np.asarray(centers)
    arr_radius  = np.asarray(radius)

    centers_inds = np.argsort(arr_centers[:,0])
    sorted_centers = arr_centers[centers_inds[::]]
    sorted_radius = arr_radius[centers_inds[::]]
    """
    print("before:", arr_centers)
    print(arr_radius)
    print("after:", sorted_centers)
    print(sorted_radius)
    """

    new_centers = [sorted_centers[i] for i in range(len(sorted_centers)) if sorted_radius[i]>1 and sorted_radius[i]<7]
    new_radius  = [sorted_radius[i] for i in range(len(sorted_radius)) if sorted_radius[i]>1 and sorted_radius[i]<7] 
    
    """
    for i, x in enumerate(new_centers):
      print("new centers", x,"new radius", new_radius[i])
    """

    """
    print(boundRect)
    """

    drawing = np.zeros((canny_output.shape[0], canny_output.shape[1], 3), dtype=np.uint8)

    """
    for i in range(len(centers)):
      color = (rng.randint(0,256), rng.randint(0,256), rng.randint(0,256))
      cv2.circle(drawing, (int(centers[i][0]), int(centers[i][1])), int(radius[i]), color, 2)
    """
    
    for i in range(len(new_centers)):
        color = (rng.randint(0,256), rng.randint(0,256), rng.randint(0,256))
        cv2.circle(drawing, (int(new_centers[i][0]), int(new_centers[i][1])), int(new_radius[i]), color, 2)
    
    """
    displayGreyImage(drawing, "contours")
    """
    
    boxes = []
    scores = []
    boxes_tensor = torch.empty(size=(len(new_centers), 4))
    scores_tensor = torch.empty(len(new_centers))
    for i, x in enumerate(new_centers):
        #print(int(x[0]-new_radius[i]))
        bottom = np.floor(x[1]-new_radius[i]).astype(int)
        top = np.ceil(x[1]+new_radius[i]).astype(int)
        left = np.floor(x[0]-new_radius[i]).astype(int)
        right = np.ceil(x[0]+new_radius[i]).astype(int)
        
        """
        print(f'bottom: {bottom} top: {top} left: {left} right: {right}')
        """

        box = src_gray[bottom:top, left:right]
        score = box.sum()
        scores.append(score)

        #displayGreyImage(box, i)
        boxes.append(box)
        """
        displayGreyImage(boxes[i], i)
        """
        #box_tensor = torch.tensor([bottom, left, top, right])
        #boxes_tensor = torch.cat([box_tensor],0)
        boxes_tensor[i] = torch.tensor([left, bottom, right, top])
        scores_tensor[i] = torch.tensor([score], dtype=torch.float32)

    
    """
    print(f"boxes:{boxes_tensor} scores: {scores_tensor}")
    """

    
    max_boxes_ind = nms(boxes = boxes_tensor, scores = scores_tensor, iou_threshold=0.1)
    
    """
    print(max_boxes_ind)
    """
    
    #cv2_imshow(drawing)
    
    j_list = np.asarray(new_centers)
    """
    print(j_list[:,0])
    """
    j_list = np.asarray(j_list[:,0])
    j_list = np.sort(j_list)
    """
    print("jlist", j_list)
    #print(j_list[1]-j_list[0])
    """

    # create a list of the diffirence in centers between consecutive values
    gaps = [y - x for x, y in zip(j_list[:-1], j_list[1:])]
    """
    print(gaps)
    """
    
    # have python calculate the standard deviation for the gaps
    #sd = stdev(gaps)
    """
    print(sd)
    """
    # create a list of lists, put the first value of the source data in the first
    lists = [[j_list[0]]]
    """
    print(lists)
    """
    radius_lists = [[new_radius[0]]]
    mean_radius = np.median(new_radius)
    #print(mean_radius)
    for i, x in enumerate(j_list[1:]):
        """
        print(x- lists[-1][-1])
        print(new_radius[i])
        
        print(new_radius[i+1])
        print(x-new_radius[i] - (lists[-1][-1]+new_radius[i-1]))
        """
        # if the gap from the current item to the previous is more than 1 SD
        # Note: the previous item is the last item in the last list
        # Note: the '> 1' is the part you'd modify to make it stricter or more relaxed
        if np.abs(x - (lists[-1][-1])) > 6: #radius[i] (new_radius[i]+new_radius[i+1])*2/3
            lists.append([])
            radius_lists.append([])
        lists[-1].append(x)
        radius_lists[-1].append(new_radius[i])
    """
    print(lists)
    print(radius_lists)
    """
    digit_centers = np.asarray([np.mean(lists[i]) for i in range(len(lists))]).astype(int)
    digit_radius = np.asarray([np.max(radius_lists[i]) for i in range(len(radius_lists))]).astype(int)

    final_boxes = create_boxes_hacks(src, digit_centers)

    """
    for i in range(len(final_boxes)):
        displayGreyImage(final_boxes[i],"")
    """
    
    n_digits = len(final_boxes)

    """
    print(digit_centers, digit_radius)
    print(digit_centers)
    print(len(digit_centers))
    """
    return final_boxes, n_digits

In [14]:
wrong = 0 
wrong_arr = []
wrong_arr_index = []
correct = []

for i, sample in enumerate(train_dataset):
    snips, n_dig = thresh_callback(sample)
    real_num_digits = 5 - list(train_labels[i]).count(10)

    if(n_dig != real_num_digits):
        print("incorrect at index:", i)
        wrong += 1
        wrong_arr.append(snips)
        wrong_arr_index.append(i)  

print("total percentage incorrect:", wrong / len(train_dataset) * 100, "%")

incorrect at index: 2903
incorrect at index: 4555
incorrect at index: 7230
incorrect at index: 7472
incorrect at index: 12016
incorrect at index: 12127
incorrect at index: 13485
incorrect at index: 16017
incorrect at index: 18127
incorrect at index: 19418
incorrect at index: 19580
incorrect at index: 19752
incorrect at index: 20317
incorrect at index: 20829
incorrect at index: 24262
incorrect at index: 25284
incorrect at index: 28261
incorrect at index: 32014
incorrect at index: 32944
incorrect at index: 32954
incorrect at index: 33012
incorrect at index: 38206
incorrect at index: 48432
incorrect at index: 50390
incorrect at index: 51039
total percentage incorrect: 0.044642857142857144 %


In [15]:
num_test_digits = 0

for i, sample in enumerate(test_dataset):
    images, n_digits = thresh_callback(sample)
   
    num_test_digits += n_digits

divs = []

for div in range(1, 100):
    if(num_test_digits % (div) == 0):
        divs.append(div)

batch_size = divs[-1]
print(batch_size)

17


# CNN

In [16]:
def create_batches(input_array, label_array, batch_size):
    batched = []
    label_batched = []
    
    for i in range(np.floor(len(input_array)/batch_size).astype(int)):
        batched.append(np.expand_dims((np.array(input_array[i*batch_size:i*batch_size+batch_size])).astype(np.single),axis=1))
        label_batched.append(label_array[i*batch_size:i*batch_size+batch_size])
        
    return np.array(batched), label_batched

In [28]:
class Net(nn.Module):
    def __init__(self, h_layers):
        self.num_layers = len(h_layers)
        self.h_layers = h_layers      
        self.convs = []
        
        super(Net, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(3 * 3 * 64, 1000)
        self.fc2 = nn.Linear(1000, 10)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.drop_out(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

# Train CNN for hyper-parameter testing

In [30]:
training_dataset_batch, training_labels_batch = create_batches(xTrainingSet, yTrainingSet, batch_size)
validation_dataset_batch, validation_labels_batch = create_batches(xValidationSet, yValidationSet, batch_size)

# model hyper-parameters
h_layers = ([batch_size, 64])
max_iters = 10
learning_rate = 0.001
momentum = 0.9

# train neural network
net = Net(h_layers)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=momentum)

for epoch in range(max_iters):
    running_loss = 0.0
    
    for i, data in enumerate(training_dataset_batch):
        inputs = torch.from_numpy(data)
        labels = training_labels_batch[i] 
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, torch.from_numpy(np.array(labels).astype(np.longlong)))
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if i % 20 == 19:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 20))
            running_loss = 0.0

# test validation set on model
correct = 0
total = 0

with torch.no_grad():
    for i, data in enumerate(validation_dataset_batch):
        images = torch.from_numpy(data)
        labels = validation_labels_batch[i]
        labels = torch.from_numpy(np.array(labels).astype(np.longlong))
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        if (int(predicted[0]) != int(labels[0])):
            None

print(f"validation set accuracy ({total} samples): {(100 * correct / total)}")

[1,    20] loss: 7.144
[1,    40] loss: 2.014
[1,    60] loss: 1.809
[1,    80] loss: 1.753
[1,   100] loss: 1.691
[1,   120] loss: 1.692
[1,   140] loss: 1.612
[1,   160] loss: 1.669
[1,   180] loss: 1.528
[1,   200] loss: 1.370
[1,   220] loss: 1.368
[1,   240] loss: 1.384
[1,   260] loss: 1.209
[1,   280] loss: 1.182
[1,   300] loss: 1.204
[1,   320] loss: 1.157
[1,   340] loss: 0.980
[1,   360] loss: 0.855
[1,   380] loss: 0.890
[1,   400] loss: 1.045
[1,   420] loss: 0.854
[1,   440] loss: 0.834
[1,   460] loss: 0.847
[1,   480] loss: 0.719
[1,   500] loss: 0.746
[1,   520] loss: 0.803
[1,   540] loss: 0.649
[1,   560] loss: 0.788
[1,   580] loss: 0.796
[1,   600] loss: 0.646
[1,   620] loss: 0.780
[1,   640] loss: 0.580
[1,   660] loss: 0.665
[1,   680] loss: 0.716
[1,   700] loss: 0.650
[1,   720] loss: 0.660
[1,   740] loss: 0.518
[1,   760] loss: 0.675
[1,   780] loss: 0.602
[1,   800] loss: 0.550
[1,   820] loss: 0.641
[1,   840] loss: 0.470
[1,   860] loss: 0.699
[1,   880] 

[1,  7160] loss: 0.234
[1,  7180] loss: 0.192
[1,  7200] loss: 0.162
[1,  7220] loss: 0.195
[1,  7240] loss: 0.170
[1,  7260] loss: 0.151
[1,  7280] loss: 0.243
[1,  7300] loss: 0.153
[1,  7320] loss: 0.169
[1,  7340] loss: 0.186
[1,  7360] loss: 0.201
[1,  7380] loss: 0.191
[1,  7400] loss: 0.228
[1,  7420] loss: 0.153
[1,  7440] loss: 0.116
[1,  7460] loss: 0.220
[1,  7480] loss: 0.199
[1,  7500] loss: 0.139
[1,  7520] loss: 0.147
[1,  7540] loss: 0.195
[1,  7560] loss: 0.177
[1,  7580] loss: 0.176
[1,  7600] loss: 0.167
[1,  7620] loss: 0.129
[1,  7640] loss: 0.173
[1,  7660] loss: 0.155
[1,  7680] loss: 0.139
[1,  7700] loss: 0.298
[1,  7720] loss: 0.157
[1,  7740] loss: 0.134
[1,  7760] loss: 0.141
[1,  7780] loss: 0.165
[1,  7800] loss: 0.182
[1,  7820] loss: 0.130
[1,  7840] loss: 0.163
[1,  7860] loss: 0.176
[1,  7880] loss: 0.256
[1,  7900] loss: 0.146
[2,    20] loss: 0.124
[2,    40] loss: 0.124
[2,    60] loss: 0.251
[2,    80] loss: 0.108
[2,   100] loss: 0.241
[2,   120] 

[2,  6400] loss: 0.100
[2,  6420] loss: 0.114
[2,  6440] loss: 0.168
[2,  6460] loss: 0.120
[2,  6480] loss: 0.172
[2,  6500] loss: 0.170
[2,  6520] loss: 0.090
[2,  6540] loss: 0.157
[2,  6560] loss: 0.100
[2,  6580] loss: 0.137
[2,  6600] loss: 0.078
[2,  6620] loss: 0.125
[2,  6640] loss: 0.178
[2,  6660] loss: 0.151
[2,  6680] loss: 0.130
[2,  6700] loss: 0.112
[2,  6720] loss: 0.148
[2,  6740] loss: 0.119
[2,  6760] loss: 0.065
[2,  6780] loss: 0.115
[2,  6800] loss: 0.133
[2,  6820] loss: 0.126
[2,  6840] loss: 0.261
[2,  6860] loss: 0.188
[2,  6880] loss: 0.129
[2,  6900] loss: 0.125
[2,  6920] loss: 0.110
[2,  6940] loss: 0.082
[2,  6960] loss: 0.108
[2,  6980] loss: 0.148
[2,  7000] loss: 0.095
[2,  7020] loss: 0.146
[2,  7040] loss: 0.105
[2,  7060] loss: 0.108
[2,  7080] loss: 0.129
[2,  7100] loss: 0.117
[2,  7120] loss: 0.130
[2,  7140] loss: 0.072
[2,  7160] loss: 0.210
[2,  7180] loss: 0.117
[2,  7200] loss: 0.130
[2,  7220] loss: 0.170
[2,  7240] loss: 0.176
[2,  7260] 

[3,  5640] loss: 0.132
[3,  5660] loss: 0.074
[3,  5680] loss: 0.098
[3,  5700] loss: 0.091
[3,  5720] loss: 0.083
[3,  5740] loss: 0.074
[3,  5760] loss: 0.082
[3,  5780] loss: 0.150
[3,  5800] loss: 0.108
[3,  5820] loss: 0.102
[3,  5840] loss: 0.099
[3,  5860] loss: 0.076
[3,  5880] loss: 0.107
[3,  5900] loss: 0.078
[3,  5920] loss: 0.084
[3,  5940] loss: 0.109
[3,  5960] loss: 0.057
[3,  5980] loss: 0.095
[3,  6000] loss: 0.161
[3,  6020] loss: 0.079
[3,  6040] loss: 0.174
[3,  6060] loss: 0.064
[3,  6080] loss: 0.049
[3,  6100] loss: 0.046
[3,  6120] loss: 0.053
[3,  6140] loss: 0.104
[3,  6160] loss: 0.127
[3,  6180] loss: 0.084
[3,  6200] loss: 0.058
[3,  6220] loss: 0.106
[3,  6240] loss: 0.184
[3,  6260] loss: 0.097
[3,  6280] loss: 0.076
[3,  6300] loss: 0.119
[3,  6320] loss: 0.130
[3,  6340] loss: 0.059
[3,  6360] loss: 0.053
[3,  6380] loss: 0.101
[3,  6400] loss: 0.147
[3,  6420] loss: 0.121
[3,  6440] loss: 0.169
[3,  6460] loss: 0.100
[3,  6480] loss: 0.094
[3,  6500] 

[4,  4880] loss: 0.084
[4,  4900] loss: 0.082
[4,  4920] loss: 0.113
[4,  4940] loss: 0.131
[4,  4960] loss: 0.138
[4,  4980] loss: 0.096
[4,  5000] loss: 0.174
[4,  5020] loss: 0.091
[4,  5040] loss: 0.099
[4,  5060] loss: 0.141
[4,  5080] loss: 0.153
[4,  5100] loss: 0.098
[4,  5120] loss: 0.178
[4,  5140] loss: 0.109
[4,  5160] loss: 0.110
[4,  5180] loss: 0.198
[4,  5200] loss: 0.116
[4,  5220] loss: 0.071
[4,  5240] loss: 0.097
[4,  5260] loss: 0.174
[4,  5280] loss: 0.089
[4,  5300] loss: 0.119
[4,  5320] loss: 0.115
[4,  5340] loss: 0.045
[4,  5360] loss: 0.104
[4,  5380] loss: 0.057
[4,  5400] loss: 0.095
[4,  5420] loss: 0.108
[4,  5440] loss: 0.081
[4,  5460] loss: 0.075
[4,  5480] loss: 0.076
[4,  5500] loss: 0.095
[4,  5520] loss: 0.033
[4,  5540] loss: 0.142
[4,  5560] loss: 0.052
[4,  5580] loss: 0.065
[4,  5600] loss: 0.094
[4,  5620] loss: 0.154
[4,  5640] loss: 0.127
[4,  5660] loss: 0.077
[4,  5680] loss: 0.063
[4,  5700] loss: 0.078
[4,  5720] loss: 0.045
[4,  5740] 

...

[7,  2600] loss: 0.154
[7,  2620] loss: 0.105
[7,  2640] loss: 0.078
[7,  2660] loss: 0.118
[7,  2680] loss: 0.062
[7,  2700] loss: 0.095
[7,  2720] loss: 0.096
[7,  2740] loss: 0.107
[7,  2760] loss: 0.056
[7,  2780] loss: 0.095
[7,  2800] loss: 0.049
[7,  2820] loss: 0.052
[7,  2840] loss: 0.088
[7,  2860] loss: 0.133
[7,  2880] loss: 0.102
[7,  2900] loss: 0.078
[7,  2920] loss: 0.039
[7,  2940] loss: 0.065
[7,  2960] loss: 0.074
[7,  2980] loss: 0.101
[7,  3000] loss: 0.135
[7,  3020] loss: 0.093
[7,  3040] loss: 0.072
[7,  3060] loss: 0.117
[7,  3080] loss: 0.100
[7,  3100] loss: 0.115
[7,  3120] loss: 0.065
[7,  3140] loss: 0.090
[7,  3160] loss: 0.111
[7,  3180] loss: 0.095
[7,  3200] loss: 0.128
[7,  3220] loss: 0.056
[7,  3240] loss: 0.104
[7,  3260] loss: 0.112
[7,  3280] loss: 0.071
[7,  3300] loss: 0.214
[7,  3320] loss: 0.060
[7,  3340] loss: 0.067
[7,  3360] loss: 0.116
[7,  3380] loss: 0.077
[7,  3400] loss: 0.148
[7,  3420] loss: 0.087
[7,  3440] loss: 0.051
[7,  3460] 

[8,  1840] loss: 0.094
[8,  1860] loss: 0.036
[8,  1880] loss: 0.060
[8,  1900] loss: 0.109
[8,  1920] loss: 0.079
[8,  1940] loss: 0.109
[8,  1960] loss: 0.082
[8,  1980] loss: 0.079
[8,  2000] loss: 0.099
[8,  2020] loss: 0.065
[8,  2040] loss: 0.231
[8,  2060] loss: 0.098
[8,  2080] loss: 0.071
[8,  2100] loss: 0.083
[8,  2120] loss: 0.092
[8,  2140] loss: 0.165
[8,  2160] loss: 0.112
[8,  2180] loss: 0.032
[8,  2200] loss: 0.084
[8,  2220] loss: 0.199
[8,  2240] loss: 0.072
[8,  2260] loss: 0.102
[8,  2280] loss: 0.154
[8,  2300] loss: 0.110
[8,  2320] loss: 0.158
[8,  2340] loss: 0.138
[8,  2360] loss: 0.072
[8,  2380] loss: 0.078
[8,  2400] loss: 0.118
[8,  2420] loss: 0.056
[8,  2440] loss: 0.114
[8,  2460] loss: 0.096
[8,  2480] loss: 0.138
[8,  2500] loss: 0.069
[8,  2520] loss: 0.062
[8,  2540] loss: 0.049
[8,  2560] loss: 0.069
[8,  2580] loss: 0.090
[8,  2600] loss: 0.102
[8,  2620] loss: 0.123
[8,  2640] loss: 0.053
[8,  2660] loss: 0.081
[8,  2680] loss: 0.095
[8,  2700] 

[9,  1080] loss: 0.089
[9,  1100] loss: 0.103
[9,  1120] loss: 0.073
[9,  1140] loss: 0.094
[9,  1160] loss: 0.054
[9,  1180] loss: 0.066
[9,  1200] loss: 0.055
[9,  1220] loss: 0.028
[9,  1240] loss: 0.076
[9,  1260] loss: 0.042
[9,  1280] loss: 0.088
[9,  1300] loss: 0.113
[9,  1320] loss: 0.029
[9,  1340] loss: 0.063
[9,  1360] loss: 0.065
[9,  1380] loss: 0.095
[9,  1400] loss: 0.146
[9,  1420] loss: 0.096
[9,  1440] loss: 0.096
[9,  1460] loss: 0.114
[9,  1480] loss: 0.070
[9,  1500] loss: 0.087
[9,  1520] loss: 0.158
[9,  1540] loss: 0.080
[9,  1560] loss: 0.071
[9,  1580] loss: 0.061
[9,  1600] loss: 0.097
[9,  1620] loss: 0.080
[9,  1640] loss: 0.045
[9,  1660] loss: 0.112
[9,  1680] loss: 0.156
[9,  1700] loss: 0.227
[9,  1720] loss: 0.048
[9,  1740] loss: 0.087
[9,  1760] loss: 0.134
[9,  1780] loss: 0.063
[9,  1800] loss: 0.100
[9,  1820] loss: 0.069
[9,  1840] loss: 0.117
[9,  1860] loss: 0.094
[9,  1880] loss: 0.057
[9,  1900] loss: 0.063
[9,  1920] loss: 0.088
[9,  1940] 

[10,   300] loss: 0.114
[10,   320] loss: 0.094
[10,   340] loss: 0.075
[10,   360] loss: 0.068
[10,   380] loss: 0.103
[10,   400] loss: 0.141
[10,   420] loss: 0.112
[10,   440] loss: 0.065
[10,   460] loss: 0.161
[10,   480] loss: 0.128
[10,   500] loss: 0.052
[10,   520] loss: 0.042
[10,   540] loss: 0.055
[10,   560] loss: 0.087
[10,   580] loss: 0.099
[10,   600] loss: 0.073
[10,   620] loss: 0.080
[10,   640] loss: 0.049
[10,   660] loss: 0.075
[10,   680] loss: 0.065
[10,   700] loss: 0.067
[10,   720] loss: 0.064
[10,   740] loss: 0.094
[10,   760] loss: 0.109
[10,   780] loss: 0.076
[10,   800] loss: 0.071
[10,   820] loss: 0.072
[10,   840] loss: 0.053
[10,   860] loss: 0.049
[10,   880] loss: 0.047
[10,   900] loss: 0.091
[10,   920] loss: 0.110
[10,   940] loss: 0.106
[10,   960] loss: 0.073
[10,   980] loss: 0.100
[10,  1000] loss: 0.091
[10,  1020] loss: 0.064
[10,  1040] loss: 0.128
[10,  1060] loss: 0.084
[10,  1080] loss: 0.077
[10,  1100] loss: 0.060
[10,  1120] loss

[10,  7140] loss: 0.064
[10,  7160] loss: 0.047
[10,  7180] loss: 0.121
[10,  7200] loss: 0.127
[10,  7220] loss: 0.074
[10,  7240] loss: 0.071
[10,  7260] loss: 0.119
[10,  7280] loss: 0.130
[10,  7300] loss: 0.129
[10,  7320] loss: 0.056
[10,  7340] loss: 0.060
[10,  7360] loss: 0.043
[10,  7380] loss: 0.055
[10,  7400] loss: 0.061
[10,  7420] loss: 0.046
[10,  7440] loss: 0.040
[10,  7460] loss: 0.064
[10,  7480] loss: 0.049
[10,  7500] loss: 0.122
[10,  7520] loss: 0.137
[10,  7540] loss: 0.055
[10,  7560] loss: 0.093
[10,  7580] loss: 0.059
[10,  7600] loss: 0.021
[10,  7620] loss: 0.019
[10,  7640] loss: 0.055
[10,  7660] loss: 0.074
[10,  7680] loss: 0.037
[10,  7700] loss: 0.144
[10,  7720] loss: 0.060
[10,  7740] loss: 0.065
[10,  7760] loss: 0.054
[10,  7780] loss: 0.032
[10,  7800] loss: 0.047
[10,  7820] loss: 0.045
[10,  7840] loss: 0.086
[10,  7860] loss: 0.037
[10,  7880] loss: 0.088
[10,  7900] loss: 0.073
validation set accuracy (33575 samples): 97.8108711839166


# Train CNN to predict unseen dataset

In [31]:
full_dataset = xTrainingSet + xValidationSet
full_labels = yTrainingSet + yValidationSet

full_dataset_batch, full_labels_batch = create_batches(full_dataset, full_labels, batch_size)

net = Net(h_layers)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=momentum)

for epoch in range(max_iters):
    running_loss = 0.0
    
    for i, data in enumerate(full_dataset_batch):
        inputs = torch.from_numpy(data)
        labels = full_labels_batch[i] 
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, torch.from_numpy(np.array(labels).astype(np.longlong)))
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if i % 20 == 19:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 20))
            running_loss = 0.0

[1,    20] loss: 8.465
[1,    40] loss: 1.774
[1,    60] loss: 1.395
[1,    80] loss: 1.256
[1,   100] loss: 1.085
[1,   120] loss: 1.071
[1,   140] loss: 0.997
[1,   160] loss: 0.883
[1,   180] loss: 0.848
[1,   200] loss: 0.742
[1,   220] loss: 0.648
[1,   240] loss: 0.717
[1,   260] loss: 0.642
[1,   280] loss: 0.667
[1,   300] loss: 0.726
[1,   320] loss: 0.692
[1,   340] loss: 0.682
[1,   360] loss: 0.552
[1,   380] loss: 0.584
[1,   400] loss: 0.518
[1,   420] loss: 0.527
[1,   440] loss: 0.411
[1,   460] loss: 0.596
[1,   480] loss: 0.495
[1,   500] loss: 0.462
[1,   520] loss: 0.453
[1,   540] loss: 0.489
[1,   560] loss: 0.662
[1,   580] loss: 0.576
[1,   600] loss: 0.414
[1,   620] loss: 0.631
[1,   640] loss: 0.489
[1,   660] loss: 0.497
[1,   680] loss: 0.372
[1,   700] loss: 0.481
[1,   720] loss: 0.440
[1,   740] loss: 0.414
[1,   760] loss: 0.477
[1,   780] loss: 0.330
[1,   800] loss: 0.351
[1,   820] loss: 0.437
[1,   840] loss: 0.391
[1,   860] loss: 0.521
[1,   880] 

[1,  7160] loss: 0.207
[1,  7180] loss: 0.138
[1,  7200] loss: 0.096
[1,  7220] loss: 0.203
[1,  7240] loss: 0.215
[1,  7260] loss: 0.132
[1,  7280] loss: 0.226
[1,  7300] loss: 0.188
[1,  7320] loss: 0.143
[1,  7340] loss: 0.168
[1,  7360] loss: 0.213
[1,  7380] loss: 0.129
[1,  7400] loss: 0.179
[1,  7420] loss: 0.139
[1,  7440] loss: 0.160
[1,  7460] loss: 0.184
[1,  7480] loss: 0.194
[1,  7500] loss: 0.216
[1,  7520] loss: 0.184
[1,  7540] loss: 0.194
[1,  7560] loss: 0.276
[1,  7580] loss: 0.140
[1,  7600] loss: 0.233
[1,  7620] loss: 0.118
[1,  7640] loss: 0.118
[1,  7660] loss: 0.170
[1,  7680] loss: 0.112
[1,  7700] loss: 0.260
[1,  7720] loss: 0.086
[1,  7740] loss: 0.098
[1,  7760] loss: 0.105
[1,  7780] loss: 0.162
[1,  7800] loss: 0.169
[1,  7820] loss: 0.117
[1,  7840] loss: 0.117
[1,  7860] loss: 0.144
[1,  7880] loss: 0.220
[1,  7900] loss: 0.157
[1,  7920] loss: 0.131
[1,  7940] loss: 0.125
[1,  7960] loss: 0.152
[1,  7980] loss: 0.225
[1,  8000] loss: 0.219
[1,  8020] 

[2,  4460] loss: 0.136
[2,  4480] loss: 0.123
[2,  4500] loss: 0.121
[2,  4520] loss: 0.163
[2,  4540] loss: 0.190
[2,  4560] loss: 0.067
[2,  4580] loss: 0.094
[2,  4600] loss: 0.090
[2,  4620] loss: 0.099
[2,  4640] loss: 0.093
[2,  4660] loss: 0.114
[2,  4680] loss: 0.070
[2,  4700] loss: 0.161
[2,  4720] loss: 0.165
[2,  4740] loss: 0.162
[2,  4760] loss: 0.077
[2,  4780] loss: 0.141
[2,  4800] loss: 0.143
[2,  4820] loss: 0.068
[2,  4840] loss: 0.112
[2,  4860] loss: 0.126
[2,  4880] loss: 0.093
[2,  4900] loss: 0.076
[2,  4920] loss: 0.112
[2,  4940] loss: 0.119
[2,  4960] loss: 0.120
[2,  4980] loss: 0.185
[2,  5000] loss: 0.169
[2,  5020] loss: 0.094
[2,  5040] loss: 0.278
[2,  5060] loss: 0.186
[2,  5080] loss: 0.124
[2,  5100] loss: 0.195
[2,  5120] loss: 0.107
[2,  5140] loss: 0.132
[2,  5160] loss: 0.087
[2,  5180] loss: 0.234
[2,  5200] loss: 0.138
[2,  5220] loss: 0.113
[2,  5240] loss: 0.099
[2,  5260] loss: 0.124
[2,  5280] loss: 0.106
[2,  5300] loss: 0.117
[2,  5320] 

[3,  1740] loss: 0.134
[3,  1760] loss: 0.155
[3,  1780] loss: 0.172
[3,  1800] loss: 0.157
[3,  1820] loss: 0.151
[3,  1840] loss: 0.155
[3,  1860] loss: 0.064
[3,  1880] loss: 0.060
[3,  1900] loss: 0.059
[3,  1920] loss: 0.085
[3,  1940] loss: 0.059
[3,  1960] loss: 0.106
[3,  1980] loss: 0.120
[3,  2000] loss: 0.104
[3,  2020] loss: 0.125
[3,  2040] loss: 0.177
[3,  2060] loss: 0.118
[3,  2080] loss: 0.060
[3,  2100] loss: 0.128
[3,  2120] loss: 0.084
[3,  2140] loss: 0.136
[3,  2160] loss: 0.098
[3,  2180] loss: 0.033
[3,  2200] loss: 0.042
[3,  2220] loss: 0.096
[3,  2240] loss: 0.113
[3,  2260] loss: 0.126
[3,  2280] loss: 0.086
[3,  2300] loss: 0.076
[3,  2320] loss: 0.143
[3,  2340] loss: 0.116
[3,  2360] loss: 0.066
[3,  2380] loss: 0.122
[3,  2400] loss: 0.165
[3,  2420] loss: 0.092
[3,  2440] loss: 0.107
[3,  2460] loss: 0.065
[3,  2480] loss: 0.170
[3,  2500] loss: 0.124
[3,  2520] loss: 0.087
[3,  2540] loss: 0.100
[3,  2560] loss: 0.119
[3,  2580] loss: 0.085
[3,  2600] 

[3,  8880] loss: 0.067
[3,  8900] loss: 0.126
[3,  8920] loss: 0.082
[3,  8940] loss: 0.146
[3,  8960] loss: 0.082
[3,  8980] loss: 0.165
[3,  9000] loss: 0.082
[3,  9020] loss: 0.133
[3,  9040] loss: 0.110
[3,  9060] loss: 0.118
[3,  9080] loss: 0.090
[3,  9100] loss: 0.096
[3,  9120] loss: 0.104
[3,  9140] loss: 0.087
[3,  9160] loss: 0.092
[3,  9180] loss: 0.112
[3,  9200] loss: 0.101
[3,  9220] loss: 0.062
[3,  9240] loss: 0.136
[3,  9260] loss: 0.083
[3,  9280] loss: 0.111
[3,  9300] loss: 0.065
[3,  9320] loss: 0.038
[3,  9340] loss: 0.095
[3,  9360] loss: 0.149
[3,  9380] loss: 0.058
[3,  9400] loss: 0.079
[3,  9420] loss: 0.086
[3,  9440] loss: 0.082
[3,  9460] loss: 0.035
[3,  9480] loss: 0.069
[3,  9500] loss: 0.115
[3,  9520] loss: 0.115
[3,  9540] loss: 0.090
[3,  9560] loss: 0.053
[3,  9580] loss: 0.080
[3,  9600] loss: 0.066
[3,  9620] loss: 0.032
[3,  9640] loss: 0.094
[3,  9660] loss: 0.081
[3,  9680] loss: 0.078
[3,  9700] loss: 0.094
[3,  9720] loss: 0.046
[3,  9740] 

...

[7,  5160] loss: 0.054
[7,  5180] loss: 0.163
[7,  5200] loss: 0.074
[7,  5220] loss: 0.071
[7,  5240] loss: 0.078
[7,  5260] loss: 0.075
[7,  5280] loss: 0.136
[7,  5300] loss: 0.064
[7,  5320] loss: 0.047
[7,  5340] loss: 0.112
[7,  5360] loss: 0.080
[7,  5380] loss: 0.077
[7,  5400] loss: 0.086
[7,  5420] loss: 0.140
[7,  5440] loss: 0.085
[7,  5460] loss: 0.038
[7,  5480] loss: 0.062
[7,  5500] loss: 0.085
[7,  5520] loss: 0.048
[7,  5540] loss: 0.117
[7,  5560] loss: 0.072
[7,  5580] loss: 0.075
[7,  5600] loss: 0.103
[7,  5620] loss: 0.099
[7,  5640] loss: 0.071
[7,  5660] loss: 0.113
[7,  5680] loss: 0.078
[7,  5700] loss: 0.085
[7,  5720] loss: 0.072
[7,  5740] loss: 0.108
[7,  5760] loss: 0.066
[7,  5780] loss: 0.065
[7,  5800] loss: 0.066
[7,  5820] loss: 0.047
[7,  5840] loss: 0.061
[7,  5860] loss: 0.067
[7,  5880] loss: 0.095
[7,  5900] loss: 0.036
[7,  5920] loss: 0.089
[7,  5940] loss: 0.068
[7,  5960] loss: 0.055
[7,  5980] loss: 0.086
[7,  6000] loss: 0.060
[7,  6020] 

[8,  2440] loss: 0.063
[8,  2460] loss: 0.023
[8,  2480] loss: 0.043
[8,  2500] loss: 0.084
[8,  2520] loss: 0.090
[8,  2540] loss: 0.059
[8,  2560] loss: 0.063
[8,  2580] loss: 0.075
[8,  2600] loss: 0.090
[8,  2620] loss: 0.108
[8,  2640] loss: 0.050
[8,  2660] loss: 0.121
[8,  2680] loss: 0.096
[8,  2700] loss: 0.104
[8,  2720] loss: 0.089
[8,  2740] loss: 0.030
[8,  2760] loss: 0.102
[8,  2780] loss: 0.111
[8,  2800] loss: 0.085
[8,  2820] loss: 0.079
[8,  2840] loss: 0.093
[8,  2860] loss: 0.160
[8,  2880] loss: 0.086
[8,  2900] loss: 0.128
[8,  2920] loss: 0.062
[8,  2940] loss: 0.078
[8,  2960] loss: 0.089
[8,  2980] loss: 0.059
[8,  3000] loss: 0.135
[8,  3020] loss: 0.159
[8,  3040] loss: 0.049
[8,  3060] loss: 0.075
[8,  3080] loss: 0.041
[8,  3100] loss: 0.132
[8,  3120] loss: 0.046
[8,  3140] loss: 0.055
[8,  3160] loss: 0.127
[8,  3180] loss: 0.073
[8,  3200] loss: 0.113
[8,  3220] loss: 0.112
[8,  3240] loss: 0.093
[8,  3260] loss: 0.132
[8,  3280] loss: 0.065
[8,  3300] 

[8,  9600] loss: 0.063
[8,  9620] loss: 0.049
[8,  9640] loss: 0.025
[8,  9660] loss: 0.061
[8,  9680] loss: 0.094
[8,  9700] loss: 0.071
[8,  9720] loss: 0.021
[8,  9740] loss: 0.047
[8,  9760] loss: 0.132
[8,  9780] loss: 0.086
[8,  9800] loss: 0.028
[8,  9820] loss: 0.090
[8,  9840] loss: 0.059
[8,  9860] loss: 0.093
[9,    20] loss: 0.064
[9,    40] loss: 0.064
[9,    60] loss: 0.053
[9,    80] loss: 0.091
[9,   100] loss: 0.071
[9,   120] loss: 0.089
[9,   140] loss: 0.047
[9,   160] loss: 0.080
[9,   180] loss: 0.117
[9,   200] loss: 0.083
[9,   220] loss: 0.024
[9,   240] loss: 0.046
[9,   260] loss: 0.056
[9,   280] loss: 0.107
[9,   300] loss: 0.074
[9,   320] loss: 0.083
[9,   340] loss: 0.105
[9,   360] loss: 0.054
[9,   380] loss: 0.138
[9,   400] loss: 0.106
[9,   420] loss: 0.064
[9,   440] loss: 0.042
[9,   460] loss: 0.053
[9,   480] loss: 0.056
[9,   500] loss: 0.042
[9,   520] loss: 0.115
[9,   540] loss: 0.073
[9,   560] loss: 0.086
[9,   580] loss: 0.093
[9,   600] 

[9,  6880] loss: 0.070
[9,  6900] loss: 0.066
[9,  6920] loss: 0.043
[9,  6940] loss: 0.133
[9,  6960] loss: 0.049
[9,  6980] loss: 0.102
[9,  7000] loss: 0.066
[9,  7020] loss: 0.051
[9,  7040] loss: 0.035
[9,  7060] loss: 0.107
[9,  7080] loss: 0.124
[9,  7100] loss: 0.070
[9,  7120] loss: 0.059
[9,  7140] loss: 0.054
[9,  7160] loss: 0.083
[9,  7180] loss: 0.109
[9,  7200] loss: 0.032
[9,  7220] loss: 0.081
[9,  7240] loss: 0.106
[9,  7260] loss: 0.069
[9,  7280] loss: 0.115
[9,  7300] loss: 0.065
[9,  7320] loss: 0.103
[9,  7340] loss: 0.063
[9,  7360] loss: 0.088
[9,  7380] loss: 0.061
[9,  7400] loss: 0.072
[9,  7420] loss: 0.089
[9,  7440] loss: 0.052
[9,  7460] loss: 0.117
[9,  7480] loss: 0.016
[9,  7500] loss: 0.129
[9,  7520] loss: 0.093
[9,  7540] loss: 0.077
[9,  7560] loss: 0.055
[9,  7580] loss: 0.176
[9,  7600] loss: 0.119
[9,  7620] loss: 0.062
[9,  7640] loss: 0.078
[9,  7660] loss: 0.068
[9,  7680] loss: 0.087
[9,  7700] loss: 0.206
[9,  7720] loss: 0.027
[9,  7740] 

[10,  3980] loss: 0.037
[10,  4000] loss: 0.084
[10,  4020] loss: 0.075
[10,  4040] loss: 0.051
[10,  4060] loss: 0.061
[10,  4080] loss: 0.027
[10,  4100] loss: 0.032
[10,  4120] loss: 0.069
[10,  4140] loss: 0.084
[10,  4160] loss: 0.062
[10,  4180] loss: 0.043
[10,  4200] loss: 0.074
[10,  4220] loss: 0.105
[10,  4240] loss: 0.017
[10,  4260] loss: 0.078
[10,  4280] loss: 0.058
[10,  4300] loss: 0.061
[10,  4320] loss: 0.090
[10,  4340] loss: 0.119
[10,  4360] loss: 0.056
[10,  4380] loss: 0.100
[10,  4400] loss: 0.065
[10,  4420] loss: 0.069
[10,  4440] loss: 0.061
[10,  4460] loss: 0.053
[10,  4480] loss: 0.045
[10,  4500] loss: 0.079
[10,  4520] loss: 0.035
[10,  4540] loss: 0.071
[10,  4560] loss: 0.026
[10,  4580] loss: 0.055
[10,  4600] loss: 0.084
[10,  4620] loss: 0.049
[10,  4640] loss: 0.095
[10,  4660] loss: 0.071
[10,  4680] loss: 0.077
[10,  4700] loss: 0.085
[10,  4720] loss: 0.113
[10,  4740] loss: 0.080
[10,  4760] loss: 0.031
[10,  4780] loss: 0.035
[10,  4800] loss

# Predict unseen dataset

In [32]:
separated_digits = []

for i, sample in enumerate(test_dataset):
    digits = []
    
    digits, n_digits = thresh_callback(sample)
        
    separated_digits.append(digits)
    
results = []

for i in range(len(test_dataset)):
    results.append([])

row = 0
col = 0
    
for i in range(int(num_test_digits / batch_size)):
    batch = []
    source = []
    count = 0
    
    while count < batch_size:
        batch.append(separated_digits[row][col])
        source.append((row, col))
        
        if(col < len(separated_digits[row]) - 1):
            col += 1
        else:
            row += 1
            col = 0
        
        count += 1
        
    batch = np.expand_dims(np.asarray(batch).astype(np.single), axis=1) 
    batch = torch.from_numpy(batch)
    
    output = net(batch)
    _, predicted = torch.max(output.data, 1)
    
    for j, pred in enumerate(predicted):
        coord = source[j]
        results[coord[0]].append(pred.item())
        
for r in results:
    pad = 5 - len(r)
    
    for i in range(pad):
        r.append(10)

In [34]:
# for i, sample in enumerate(test_dataset):
#     displayGreyWindows(sample, "")
#     print(results[i])

In [35]:
#print(str(results[0]))

Id = []
label = []
for i, x in enumerate(results):
    Id.append(i)
    string = ''.join([str(elem) for elem in x])
    label.append(string)
"""
print(Id[0],label[0])
print(df["Label"])
"""
data={"Label":label}

df = pd.DataFrame(data)

df.to_csv('sample.csv', index_label = "Id")
df.head()



,Label
0,910101010
1,017310
2,14083
3,010101010
4,467810
